In [1]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=3165c2a2af8e4c487e5792fa21a058464ecb0654066af686b6b845504d5c6a2c
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
print(len(df))

44023


In [4]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    
    try:
      coverpage=soup.find_all('div',class_='headline-article')
      title=coverpage[0].find('h1').get_text()
      try:
       description=coverpage[0].find('h2').get_text()
      except:
        description=''
    except:
      title=''
      description=''
    
    try:
      coverpage=soup.find_all('div',class_='article-entry-author')
      time=coverpage[0].find('span',class_='article-author__date').get_text()
    except:
      time=''  

    try:
      coverpage=soup.find_all('div',class_='article-body')
      article=''
      for i in coverpage[0].find_all('p'):
        article=article+i.get_text()
      # print(article)
    except:
      article=''
      print('article error\n'+url)


  except:
    title=''
    description=''
    time=''
    article=''
    print('url error\n'+url)    

  return [title,description,time,article]

In [6]:
get_articles('https://www.eldia.es/sociedad/2022/07/27/temperaturas-descenso-oleaje-jueves-canarias-tiempo-71851227.html')

['Temperaturas en descenso y oleaje este jueves en Canarias',
 'Las máximas caerán hasta los 32 grados con el Archipiélago en prealerta por el fuerte calor',
 '\n                    27·07·22\n            ',
 '\n        Las temperaturas máximas caen este jueves en Canarias hasta los 30-32 º C, parámetros que sólo se esperan en las medianías del sur de Gran Canaria, en una jornada en la que habrá cielos despejados y vientos del nordeste con intervalos muy fuertes de madrugada en el sur de las islas, según la Agencia Estatal de Meteorología (Aemet).\n\nLa alerta\xa0por temperaturas máximas ha finalizado,\xa0permaneciendo en situación de prealerta\xa0todo el Archipiélago, a partir de las 12.00 horas de este miércoles, 27 de julio.\n\n        En el mar habrá noreste 5 o 6, arreciando a 6 o 7 mar adentro en el noroeste y sureste, excepto 4 o 5 en el nordeste. Fuerte marejada, aumentando a gruesa mar adentro en el sureste. En el oeste y suroeste, variable 1 a 4 y rizada o marejadilla.\n27/07 

In [8]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

output=df['rows'].parallel_apply(get_articles)

df['country']='Regional'
df['title']=''
df['description']=''
df['time']=''
df['article']=''

for i in range(0,len(df)):
  df.loc[i]['title','description','time','article']=output[i]

df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
str_name='/content/drive/MyDrive/Colab Notebooks/url_article.csv'
print(df)
df.to_csv(str_name, index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


url error
error
article error
https://www.eldia.es//sucesos/2021/06/20/rescatan-45-subsaharianos-8-ninos-53980695.html
article error
https://www.eldia.es/https://www.epe.es/es/sanidad/20211127/comunidades-prescinden-mitad-sanitarios-contratados-12906441
article error
https://www.eldia.es/https://www.epe.es/es/internacional/20220306/davide-piano-guerras-ucrania-13328166
article error
https://www.eldia.es/https://www.epe.es/es/politica/20220802/dos-anos-ultimo-exilio-rey-juan-carlos-abu-dabi-14209598
article error
https://www.eldia.es/https://www.epe.es/es/madrid/20220328/almeida-daoiz-velarde-polideportivo-refugiados-ucrania-13442965
article error
https://www.eldia.es/https://www.epe.es/es/internacional/20220604/estonia-rusia-otan-guerra-ucrania-13770739
article error
https://www.eldia.es/https://www.epe.es/es/andalucia/20211125/kilometros-lucha-obrera-trabajadores-amurrio-cadiz-protestas-12898407
article error
https://www.eldia.es/https://www.epe.es/es/actualidad/20220303/80-horas-furg